***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h1 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:50px;
               font-style:bold;
               color:white;">
        Part V
    </h1>
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Crusader’s Journey in the World of Images Blog Series
    </h2>
</div>

***
by : JP Fabrero

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Importing Libraries
    </h2>
</div>

***

In [1]:
import os
os.environ['SKIMAGE_DATADIR'] = '/tmp/.skimage_cache'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import skimage.io as skio
from skimage.io import imread, imshow
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage.color import rgb2hsv

from pickling import *
from pyjanitor import auto_toc
toc = auto_toc()

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-pq_lmuz6 because the default path (/home/jfabrero/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Preface
    </h2>
</div>

***
Now, we leverage on our knowledge about pixel values and explore to utilize them using different methods. Specifically, we're going to use the color values represented in the images to perform image segmentation.

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Thresholding
    </h2>
</div>

***
Thresholding is a basic image processing technique used to convert a grayscale or color image into a binary image. It involves comparing the intensity values of pixels in an image to a specified threshold value and assigning them to either one of two classes: foreground or background. If a pixel's intensity value exceeds the threshold, it is assigned to the foreground class; otherwise, it is assigned to the background class.

In [2]:
image = imread('../mega_swampert.png')
fig, ax = plt.subplots(figsize=(15,8))
imshow(image)
# ax.axis('off')
toc.add_fig('Sample Image (Mega Swampert)[1]', width=100)

<div class="header" style="
    padding: 10px;
    background: black;">
    <h3 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:15px;
               font-style:bold;
               color:white;">
        Heuristics
    </h3>
</div>

***
Some of the most basic ways to perform thresholding is simply set the threshold to 0.5 or the mean value of the image.

In [3]:
def plot_thresh(image):
    """Plot the binarized image"""
    fig, ax = plt.subplots(1, 2, figsize=(15,7))
    ax[0].imshow(image > 0.5, 'gray')
    ax[0].axis('off')
    ax[0].set_title('Threshold = 0.5')
    
    ax[1].imshow(image > image.mean(), 'gray')
    ax[1].axis('off')
    ax[1].set_title('Threshold = Mean')
    
    toc.add_fig('Sample Binarization', width=100)

In [4]:
plot_thresh(rgb2gray(image[:,:,:3]))

<div class="header" style="
    padding: 10px;
    background: black;">
    <h3 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:15px;
               font-style:bold;
               color:white;">
        Otsu's Method
    </h3>
</div>

***
Otsu's method, also known as Otsu's thresholding or Otsu's algorithm, calculates an optimal threshold value that separates the image into foreground and background based on the histogram of intensity values. Otsu's method maximizes the between-class variance, aiming to minimize the intra-class variance.

In [5]:
def plot_otsu(image):
    """Plot the binarized image"""
    thresh = threshold_otsu(image)
    img_binary = image > thresh
    plt.imshow(img_binary, 'gray')
    plt.axis('off')
    toc.add_fig('Sample Implementation of Otsu\'s Method')

In [6]:
plot_otsu(rgb2gray(image[:,:,:3]))

***
<div class="header" style="
    padding: 20px;
    background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Color Segmentation
    </h2>
</div>

***
Color segmentation involves dividing an image into distinct regions based on the color properties of pixels. It can be performed using various color spaces, such as RGB or HSV, and typically involves defining thresholds or ranges for each color channel to separate different color regions.

<div class="header" style="
    padding: 10px;
    background: black;">
    <h3 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:15px;
               font-style:bold;
               color:white;">
        RGB Color Space
    </h3>
</div>

***
RGB is the most common color model used to represent color images on electronic displays, such as computer monitors and televisions. In this model, each pixel in an image is represented by three color channels: red, green, and blue. The intensity values of these channels range from 0 (minimum intensity) to 255 (maximum intensity) in an 8-bit representation. By combining different intensities of red, green, and blue, it is possible to create various colors.

In [7]:
def rgb_blue_segmentation(image):
    """Perform image segmentation and isolate blue"""
    img_gs = ((np.stack([rgb2gray(image[:,:,:3])] * 3, axis=-1) * 600)
              .astype('int').clip(0, 255))
    
    blue_mask = (
        (image[:, :, 2] - np.maximum(image[:, :, 0], image[:, :, 1]) > 20)
        & (np.argmax(image[:,:,:3], 2) == 2)
    )
    
    img_blue = img_gs.copy()
    img_blue[blue_mask] = image[:,:,:3][blue_mask]
    
    fig, ax = plt.subplots(1, 3, figsize=(15, 6))
    ax[0].imshow(img_gs)
    ax[0].axis('off')
    ax[0].set_title('Grayscaled Image')
    
    ax[1].imshow(image[:,:,2], 'Blues')
    ax[1].axis('off')
    ax[1].set_title('Blue Channel')
    
    ax[2].imshow(img_blue)
    ax[2].axis('off')
    ax[2].set_title('Blue Segmented Image w/ Grayscale')
    
    toc.add_fig('Color Segmentation in RGB Space (Blue)', width=100)

In [8]:
rgb_blue_segmentation(image)

<div class="header" style="
    padding: 10px;
    background: black;">
    <h3 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:15px;
               font-style:bold;
               color:white;">
        HSV Color Space
    </h3>
</div>

***
It is a color model that represents colors based on three components: Hue, Saturation, and Value. 
 
Hue refers to the pure color component of a color. It represents the dominant wavelength of light and is often described in terms of common color names such as red, blue, or green. 
 
Saturation represents the intensity or purity of a color. It refers to the amount of gray added to the hue. Saturation is typically represented as a percentage, ranging from 0% (completely desaturated, grayscale) to 100% (fully saturated).

Value represents the brightness or lightness of a color. It determines how light or dark a color appears. Higher value values correspond to brighter colors, while lower value values correspond to darker colors.

In [9]:
def hsv_blue_segmentation(image):
    """Perform image segmentation and isolate blue"""
    img_gs = ((np.stack([rgb2gray(image[:,:,:3])] * 3, axis=-1) * 600)
              .astype('int').clip(0, 255))
    
    img_hsv = rgb2hsv(image[:,:,:3])
    
    lower_mask = img_hsv[:, :, 0] > 0.50
    upper_mask = img_hsv[:, :, 0] < 0.75
    blue_mask = lower_mask * upper_mask
    
    img_blue = img_gs.copy()
    img_blue[blue_mask] = image[:,:,:3][blue_mask]
    
    fig, ax = plt.subplots(1, 3, figsize=(15, 6))
    ax[0].imshow(img_gs)
    ax[0].axis('off')
    ax[0].set_title('Grayscaled Image')
    
    ax[1].imshow(img_hsv[:,:,0], 'hsv')
    ax[1].axis('off')
    ax[1].set_title('Hue Channel')
    
    ax[2].imshow(img_blue)
    ax[2].axis('off')
    ax[2].set_title('Blue Segmented Image w/ Grayscale')
    
    toc.add_fig('Color Segmentation in HSV Space (Blue)', width=100)

In [10]:
hsv_blue_segmentation(image)

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Author's Notes
    </h2>
</div>

***
Thresholding is perhaps the most universal and basic method but building up your knowledge or intuition of setting the thresholds is of utmost importance. Color segmentation opens up a lot of applications across different requirements. In this concept, what really helped was my understanding of the RGB color space. If you will notice, the RGB blue mask was done simply by making sure that the Blue Channel has the highest value and that it greater than a set amount. No need to iterate different values for Red and Green Channels. Make it a practice to integrate your learnings.

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        References and Acknowledgement
    </h2>
</div>

***
The concepts discussed were derived from our MSDS2023 - IIP Course. ChatGPT was used for concept reviews and writing prompts.

[1] Mega Swampert -Mega Evolve Art by Tomycase on DeviantArt. (2014, June 12). DeviantArt. https://www.deviantart.com/tomycase/art/Mega-Swampert-Mega-Evolve-Art-460315184